# **Random Forest with RandomizedSearchCV**
# **CMPT 459 Course Project**

This notebook demonstrates **Random Forest classification** with **hyperparameter tuning** using RandomizedSearchCV.

**Reference**: `classification_analysis.py`, `random_forest.py`


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV, KFold
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, roc_curve, roc_auc_score
)
from random_forest import RandomForest

## **1. Data Preprocessing**


In [3]:
def load_and_preprocess(path):
    df = pd.read_csv(path)
    print(f"Original shape: {df.shape}")
    df = df.replace("?", np.nan)
    threshold = 0.4 * len(df)
    df = df.dropna(thresh=threshold, axis=1)
    for col in df.select_dtypes(include="object").columns:
        df[col] = df[col].fillna("Unknown")
    df["readmitted_binary"] = df["readmitted"].map({"NO": 0, ">30": 1, "<30": 1})
    cat_cols = df.select_dtypes(include="object").columns.drop(["readmitted"], errors="ignore")
    le = LabelEncoder()
    for col in cat_cols:
        if df[col].nunique() < 10:
            df[col] = le.fit_transform(df[col].astype(str))
        else:
            df = pd.get_dummies(df, columns=[col], drop_first=True, prefix=col[:10])
    for col in ["encounter_id", "patient_nbr", "readmitted"]:
        if col in df.columns:
            df = df.drop(columns=[col])
    X = df.drop(columns=["readmitted_binary"])
    y = df["readmitted_binary"]
    num_cols = X.select_dtypes(include=["int64", "float64"]).columns
    scaler = StandardScaler()
    X[num_cols] = scaler.fit_transform(X[num_cols])
    print(f"Final shape: {X.shape}")
    return X, y

X, y = load_and_preprocess("data/diabetic_data.csv")

Original shape: (101766, 50)
Final shape: (101766, 2389)


## **2. Train/Test Split**


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"Training: {X_train.shape}, Test: {X_test.shape}")

Training: (81412, 2389), Test: (20354, 2389)


## **3. Baseline Random Forest**


In [ ]:
baseline_rf = RandomForest(n_estimators=20, max_depth=10, random_state=42)
baseline_rf.fit(X_train, y_train)
y_pred_base = baseline_rf.predict(X_test)
y_proba_base = baseline_rf.predict_proba(X_test)
acc_base = accuracy_score(y_test, y_pred_base)
f1_base = f1_score(y_test, y_pred_base, average="weighted")
auc_base = roc_auc_score(y_test, y_proba_base[:, 1])
print(f"Baseline - Accuracy: {acc_base:.4f}, F1: {f1_base:.4f}, AUC: {auc_base:.4f}")

## **4. RandomizedSearchCV**


In [ ]:
param_dist = {
    "n_estimators": [10, 20, 50, 100, 200],
    "max_depth": [5, 10, 15, 20, 30, None],
    "min_samples_split": [2, 5, 10, 15],
    "max_features": ["sqrt", "log2", None],
    "criterion": ["gini", "entropy"]
}
print("Running RandomizedSearchCV...")
random_search = RandomizedSearchCV(
    RandomForest(random_state=42),
    param_distributions=param_dist,
    n_iter=20, cv=5, scoring="f1_weighted",
    random_state=42, n_jobs=-1, verbose=1
)
random_search.fit(X_train, y_train)
print("\nBest parameters:")
for k, v in random_search.best_params_.items():
    print(f"  {k}: {v}")
print(f"Best CV F1: {random_search.best_score_:.4f}")

## **5. Tuned Model Evaluation**


In [ ]:
best_rf = random_search.best_estimator_
y_pred_tuned = best_rf.predict(X_test)
y_proba_tuned = best_rf.predict_proba(X_test)
acc_tuned = accuracy_score(y_test, y_pred_tuned)
f1_tuned = f1_score(y_test, y_pred_tuned, average="weighted")
auc_tuned = roc_auc_score(y_test, y_proba_tuned[:, 1])
print(f"Tuned - Accuracy: {acc_tuned:.4f}, F1: {f1_tuned:.4f}, AUC: {auc_tuned:.4f}")
print(f"\nImprovement:")
print(f"  Accuracy: {acc_tuned - acc_base:+.4f}")
print(f"  F1: {f1_tuned - f1_base:+.4f}")
print(f"  AUC: {auc_tuned - auc_base:+.4f}")

## **6. Visualizations**


In [ ]:
# Comparison plot
metrics = ["Accuracy", "F1 Score", "AUC-ROC"]
baseline_vals = [acc_base, f1_base, auc_base]
tuned_vals = [acc_tuned, f1_tuned, auc_tuned]
x = np.arange(len(metrics))
width = 0.35
fig, ax = plt.subplots(figsize=(10, 6))
ax.bar(x - width/2, baseline_vals, width, label="Baseline", color="steelblue")
ax.bar(x + width/2, tuned_vals, width, label="Tuned", color="darkorange")
ax.set_ylabel("Score")
ax.set_title("Baseline vs Tuned Model")
ax.set_xticks(x)
ax.set_xticklabels(metrics)
ax.legend()
ax.set_ylim([0, 1])
plt.tight_layout()
plt.show()

In [ ]:
# Confusion matrices
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
cm_base = confusion_matrix(y_test, y_pred_base)
sns.heatmap(cm_base, annot=True, fmt="d", cmap="Blues", ax=axes[0])
axes[0].set_title(f"Baseline (Acc={acc_base:.4f})")
cm_tuned = confusion_matrix(y_test, y_pred_tuned)
sns.heatmap(cm_tuned, annot=True, fmt="d", cmap="Oranges", ax=axes[1])
axes[1].set_title(f"Tuned (Acc={acc_tuned:.4f})")
plt.tight_layout()
plt.show()

In [ ]:
# ROC curves
fpr_base, tpr_base, _ = roc_curve(y_test, y_proba_base[:, 1])
fpr_tuned, tpr_tuned, _ = roc_curve(y_test, y_proba_tuned[:, 1])
plt.figure(figsize=(8, 6))
plt.plot(fpr_base, tpr_base, label=f"Baseline (AUC={auc_base:.4f})", color="steelblue")
plt.plot(fpr_tuned, tpr_tuned, label=f"Tuned (AUC={auc_tuned:.4f})", color="darkorange")
plt.plot([0, 1], [0, 1], "k--", label="Random")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve Comparison")
plt.legend()
plt.grid(alpha=0.3)
plt.show()

## **Interpretation & Discussion**

### **RandomizedSearchCV Benefits**
* Faster than GridSearchCV (samples random combinations)
* Often finds good solutions quickly
* Built-in cross-validation

### **Key Hyperparameters**
* **n_estimators**: More trees → better performance (up to a point)
* **max_depth**: Controls complexity and overfitting
* **min_samples_split**: Higher → more pruning
* **max_features**: Controls randomness in feature selection

### **Typical Improvements**
* Accuracy: +0.5-2%
* F1 Score: +1-3%
* AUC-ROC: +0.01-0.03

### **Random Forest Strengths**
* Handles non-linear relationships
* Robust to outliers
* Provides feature importances
* Less prone to overfitting

### **Best Practices**
1. Start with baseline (default parameters)
2. Use RandomizedSearchCV for tuning
3. Always cross-validate
4. Analyze feature importances
5. Monitor for overfitting
